# Fine tune CDLA pretrained model for KR court decisions

In [1]:
import subprocess, os, time, json
import fitz
import paddle
paddle.utils.run_check()

!export CUDA_VISIBLE_DEVICES=0

print(os.getcwd())

Running verify PaddlePaddle program ... 


I0515 18:23:07.056653 2191296 program_interpreter.cc:212] New Executor is Running.
W0515 18:23:07.056973 2191296 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0515 18:23:07.057626 2191296 gpu_resources.cc:164] device: 0, cuDNN Version: 8.8.


PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/home/lstm/github/PaddleDetection/tuning


I0515 18:23:07.307837 2191296 interpreter_util.cc:624] Standalone Executor is Used.


## Eval

In [11]:

command = (
    "python3 ../tools/eval.py",
    "-c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml",
    # "--slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml",
    # "-o weights=../output/best_model",
    "-o weights=./trained_weights/output_20240503/best_model/model.pdparams",
    # "use_gpu=true",
    # "--use_vdl=True --vdl_log_dir=vdl_log"
    )
command = " ".join(command)
print(command)

python3 ../tools/eval.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml -o weights=./trained_weights/output_20240503/best_model/model.pdparams


In [ ]:
!{command}

## Test 
##### Strategy: use command line method to test trained weights; create inference model for production.

#### Test prep (convert PDFs to images)

In [ ]:
# convert pdf to image and save

import fitz
import os

pdf_folder = './testdata/'
# pdf_folder = './tuning/testdata/'

for pdf_path in os.listdir(pdf_folder):

    if not pdf_path.endswith('.pdf'):
        continue

    pdf_name = os.path.splitext(pdf_path)[0]
    img_folder = os.path.join(pdf_folder, pdf_name)
    if not os.path.isdir(img_folder):
        os.mkdir(img_folder)

    pdf_path = os.path.join(pdf_folder, pdf_path)
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        img_name = os.path.join(img_folder, f'{page_num}.png')
        if not os.path.isfile(img_name):
            page.get_pixmap().save(img_name)
        else:
            print(f"{img_name} exists!")

    doc.close()

#### Test inference

In [15]:
configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml"
slim_configfile = " ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml"
# opts = "weights=./trained_weights/output_20240423/best_model/model.pdparams"

inference_datafolder = './testdata'
testdoc_img_folder = 'image1'

inference_outputfolder = './testoutput'
testout_img_folder = 'image1'

In [18]:
args = {
    'config': configfile,
    # 'slim_config': slim_configfile,
    'infer_dir': f'{inference_datafolder}/{testdoc_img_folder}',
    'infer_img': None,
    'output_dir': f'{inference_datafolder}/{testout_img_folder}',
    'draw_threshold': 0.5, # default
    # 'use_vdl': True,
    # 'vdl_log_dir': '/vdl_log_test',
    'save_results': False,
    # slice_infer, action='store_true',
    # slice_size, nargs='+,
    # overlap_ratio, nargs='+',
    # combine_method: None,
    # match_threshold: None,
    # match_metric: None,
    'visualize': True
}

print(json.dumps(args, indent=4))

{
    "config": "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml",
    "infer_dir": "./testdata/image1",
    "infer_img": null,
    "output_dir": "./testdata/image1",
    "draw_threshold": 0.5,
    "save_results": false,
    "visualize": true
}


In [ ]:
# datapath = 'testdata'
# filelist = os.listdir(datapath)

# def pdf2byte(filename):
#     pdf = fitz.open(os.path.join(datapath, filename))
#     for pg in range(pdf.pageCount):
#         # convert pdf to image and store in local variable
#         page = pdf[pg]
#         pix = page.getPixmap()
#         img_data = page.getPixmap().tobytes("png")


In [31]:
configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml"
slim_configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml"

inference_datafolder = './testdata'
testdoc_img_folder = 'JPEG_SAMPLE'
weight2use = '20240503'
output_dir = './testresults'

command = (
    "python3 ../tools/infer.py",
    f"-c {configfile}",
    # f"--slim_config {slim_configfile}",
    f"--infer_dir={inference_datafolder}/{testdoc_img_folder}",
    # f"--infer_img={inference_datafolder}/{testdoc_img_folder}/0.png",
    f"-o weights=./trained_weights/output_{weight2use}/best_model/model.pdparams",
    "--draw_threshold=0.5",
    "--save_results=true",
    "--visualize=True",
    # "use_gpu=true",
    # "--use_vdl=True",
    # "--vdl_log_dir=./vdl_log_test",
    f"--output_dir={output_dir}"
    )

command = " ".join(command)
print(command)

!{command}

python3 ../tools/infer.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml --infer_dir=./testdata/JPEG_SAMPLE -o weights=./trained_weights/output_20240503/best_model/model.pdparams --draw_threshold=0.5 --save_results=true --visualize=True --output_dir=./testresults
/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)
W0515 19:07:35.222344 2235026 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0515 19:07:35.223192 2235026 gpu_resources.cc:164] device: 0, cuDNN Version: 8.8.
[05/15 19:07:35] ppdet.utils.checkpoint INFO: Finish loading model weights: ./trained_weights/output_20240503/best_model/model.pdparams
[05/15 19:07:35] train INFO: Found 2 inference images in total.
loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
cre

## Deploy (use PaddleOCR)

#### Model export (distilled model)
https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.7/ppstructure/docs/quickstart_en.md

In [42]:
model_outpath = './exported_model/latest'
# model_outpath = './exported_model/krcase_layout_train'

command = (
    "python3 ../tools/export_model.py",
    f"-c {configfile}",
    # f"--slim_config {slim_configfile}",
    f"-o weights=./trained_weights/output_{weight2use}/best_model/model.pdparams",
    f"--output_dir={model_outpath}"
    )
command = " ".join(command)
print(command)


python3 ../tools/export_model.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml -o weights=./trained_weights/output_20240503/best_model/model.pdparams --output_dir=./exported_model/latest


In [43]:
!{command}

/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)
[05/15 20:04:15] ppdet.utils.checkpoint INFO: Finish loading model weights: ./trained_weights/output_20240503/best_model/model.pdparams
loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
[05/15 20:04:15] ppdet.engine INFO: Export inference config file to ./exported_model/latest/krcase_layout_train/infer_cfg.yml
I0515 20:04:19.211127 2296309 program_interpreter.cc:212] New Executor is Running.
[05/15 20:04:19] ppdet.engine INFO: Export model and saved in ./exported_model/latest/krcase_layout_train


#### Model inference (distilled model)

| Field | Description | Default |
|-|-|-|  
| output | result save path | ./output/table |
| table_max_len | long side of the image resize in table structure model | 488 |
| table_model_dir | Table structure model inference model path | None |
| table_char_dict_path | The dictionary path of table structure model | ../ppocr/utils/dict/table_structure_dict.txt |
| merge_no_span_structure | In the table recognition model, whether to merge '<td>' and '</td>' | False |
| layout_model_dir | Layout analysis model inference model path | None |
| layout_dict_path | The dictionary path of layout analysis model | ../ppocr/utils/dict/layout_publaynet_dict.txt |
| layout_score_threshold | The box threshold path of layout analysis model | 0.5 |
| layout_nms_threshold | The nms threshold path of layout analysis model | 0.5 |
| kie_algorithm | kie model algorithm | LayoutXLM |
| ser_model_dir | Ser model inference model path | None |
| ser_dict_path | The dictionary path of Ser model | ../train_data/XFUND/class_list_xfun.txt |
| mode | structure or kie | structure |
| image_orientation | Whether to perform image orientation classification in forward | False |
| layout | Whether to perform layout analysis in forward | True |
| table | Whether to perform table recognition in forward | True |
| ocr | Whether to perform ocr for non-table areas in layout analysis. When layout is False, it will be automatically set to False | True |
| recovery | Whether to perform layout recovery in forward | False |
| save_pdf | Whether to convert docx to pdf when recovery | False |
| structure_version | Structure version, optional PP-structure and PP-structurev2 | PP-structure |


In [35]:
import os
import cv2
import matplotlib.pyplot as plt
from paddleocr import PPStructure,draw_structure_result,save_structure_res
from paddleocr import PaddleOCR

from PIL import Image
from IPython.display import display


inference_datafolder = './testdata/JPEG_SAMPLE'
# testdoc_img_folder = 'CONVERTED_JPEG'

inference_outputfolder = './testresults_deployment/latest'
# testout_img_folder = 'CONVERTED_JPEG'

font_path = '../NotoSansKR-Regular.otf' # PaddleOCR

imgs = os.listdir(f"{inference_datafolder}")
imgs

['adobe_table_광주지법_2019구합10788_판결서_0.jpg', 'adobe_table_서울고등법원_2002노311_3.jpg']

In [44]:

# get layout

run_OCR = False #
# kr_dict_path = '../ppocr/utils/dict/korean_dict.txt'

layout_engine = PPStructure(
    # lang='korean',
    output=inference_outputfolder,
    merge_no_span_structure=False,
    # [PaddleOCR] ocr_version
    # [PaddleOCR]use_onnx
    det_model_dir=None,
    # det_algorithm #[DB]
    rec_model_dir=None,
    # [PaddleOCR] rec_algorithm #["CRNN", "SVTR_LCNet"]
    # rec_char_dict_path=kr_dict_path, # Path(__file__).parent / rec_model_config["dict_path"]
    # [PaddleOCR] cls_model_dir
    # [PaddleOCR] rec_image_shape # if ocr_version in ["PP-OCRv3", "PP-OCRv4"]
    table_model_dir=None,
    table_max_len=None, # 488
    table_char_dict_path=None,
    layout_model_dir="./exported_model/latest/krcase_layout_train", #None
    # layout_model_dir="./krcase_layout_train", #None
    # layout_model_dir="./exported_model/picodet_lcnet_x1_0_fgd_layout_infer",
    # layout_model_dir="./exported_model/krcase_layout_train_20240503",
    layout_dict_path=None, # use <layout_dict_path> ? (dict should be in exported model)
    layout_score_threshold=0.4, # 0.5
    layout_nms_threshold= 0.3, # 0.5
    kie_algorithm=None, # LayoutXLM
    ser_model_dir=None,
    ser_dict_path=None, #../train_data/XFUND/class_list_xfun.txt
    mode="structure", # ocr
    image_orientation=False,
    layout=True,
    table=False,
    ocr=run_OCR,
    recovery=False,
    save_pdf=False,
    structure_version="PP-StructureV2", #"PP-Structure"
    show_log=True,
    kwargs={"OCR_det_language":'ml', "OCR_rec_language":'korean'}
    )

for imgpath in imgs:

    # imgpath = 'adobe_table_서울고등법원_2002노311_10.jpg'
    # imgpath = 'scan_fn_부산지법_2018나55364_판결서_2.jpg'
    # imgpath = 'scan_서울중앙지법_2014가합38499_판결서_1.jpg'
    # imgpath = 'adobe_table_광주지법_2019구합10788_판결서_1.jpg'
    # imgpath = 'adobe_특허법원 2017허8459_8.jpg'

    img = cv2.imread(f"{inference_datafolder}/{imgpath}")
    # display(img)
    result = layout_engine(img)
    save_structure_res(result, inference_outputfolder, os.path.basename(imgpath).split('.')[0])

    im_show = draw_structure_result(img, result,font_path=font_path)
    imgbyte = Image.fromarray(im_show)

    # create output folder if not existing
    if not os.path.isdir(f"{inference_outputfolder}"):
        os.mkdir(f"{inference_outputfolder}")


    # show left half only (if no OCR is run)
    if not run_OCR:
        height, width = im_show.shape[:2]
        mid_x = width // 2

        # Crop left side
        box = (0, 0, mid_x, height)
        img_left = imgbyte.crop(box)

        img_left.save(f"{inference_outputfolder}/{imgpath}")

        # display image
        display(img_left)
    else:
        imgbyte.save(f"{inference_outputfolder}/{imgpath}")

        # display image
        display(imgbyte)

    # if imgpath == imgs[0]:
    #     break

    break

[2024/05/15 20:05:04] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir=None, cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/lstm/.paddleocr/whl/det/ch/Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_val

ValueError: (InvalidArgument) The input of Op(Conv) should be a 4-D or 5-D Tensor. But received: input's dimension is -1, input's shape is [].
  [Hint: Expected in_dims.size() == 4 || in_dims.size() == 5 == true, but received in_dims.size() == 4 || in_dims.size() == 5:0 != true:1.] (at /paddle/paddle/phi/kernels/cpu/conv_util.h:137)
  [operator < fused_conv2d_add_act > error]

In [ ]:
# # usin predict_system.py
# from

# python3 predict_system.py --layout_model_dir=inference/picodet_lcnet_x1_0_fgd_layout_cdla_infer \
#                           --image_dir=../DATA/sample_data/판례/png/image61_0.png \
#                           --output=../output \
#                           --table=false \
#                           --ocr=false \
#                           --vis_font_path=../NotoSansKR-Regular.otf \
#                           --layout_dict_path ../ppocr/utils/dict/layout_dict/layout_cdla_dict.txt